In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clickthrough-rate-prediction/ad_10000records.csv
/kaggle/input/clickthrough-rate-prediction/ad.csv
/kaggle/input/clickthrough-rate-prediction/new_ad.csv
/kaggle/input/clickthrough-rate-prediction/ad (1).csv


In [2]:
!pip install pyffm

In [3]:

nRowsRead = None # specify 'None' if want to read whole file
# Apply_Rate_2019.csv has 1200890 rows in reality, but we are only loading/previewing the first 1000 rows
data = pd.read_csv('/kaggle/input/clickthrough-rate-prediction/ad.csv', delimiter=',', nrows = nRowsRead)
data.dataframeName = 'Apply_Rate_2019.csv'
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 50000 rows and 10 columns


In [4]:
data.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Timestamp,Clicked on Ad
0,78.84,30.0,68448.94,161.79,Front-line even-keeled website,Silvaton,Male,Peru,2016-04-04 03:57:48,1
1,60.23,35.0,66815.54,216.24,Front-line fresh-thinking open system,West Jeremyside,Male,Papua New Guinea,2016-06-18 16:02:34,0
2,46.20,49.0,62109.80,135.24,Enhanced maximized access,Lake Vanessa,Male,Chile,2016-06-26 07:01:47,1
3,87.85,31.0,50055.33,134.46,Total zero administration software,Port Sherrystad,Male,French Polynesia,2016-04-18 21:07:28,1
4,64.38,37.0,55121.65,232.54,Devolved regional moderator,South Patrickfort,Female,Bosnia and Herzegovina,2016-07-18 18:33:05,0


In [ ]:
data=data[['Daily Time Spent on Site', 'Age', 'Area Income',
       'Daily Internet Usage', 'Ad Topic Line', 'City', 'Gender', 
        'Clicked on Ad']]

In [ ]:
label='Clicked on Ad'

In [ ]:
data.head(5)

In [ ]:
cat_features=['Ad Topic Line','City','Gender']

In [ ]:
continue_var=[ 'Daily Time Spent on Site', 'Age', 'Area Income','Daily Internet Usage']

In [ ]:

fill_mean = lambda x: x.fillna(x.mean())

for col in continue_var:
    data[col] = data[col].fillna(data[col].mean())
    data[col] = data[col].astype('float64')
data = data.fillna('-1', )

In [ ]:
from sklearn.preprocessing import LabelEncoder ,MinMaxScaler
mms = MinMaxScaler(feature_range=(0, 1))
data[continue_var] = mms.fit_transform(data[continue_var])


In [ ]:
train=data

In [ ]:
from pyffm import PyFFM

training_params = {'epochs': 5, 'reg_lambda': 0.002, "sigmoid": True,
            "parallel": True,
            "early_stop": True,}
model = PyFFM(model='ffm', training_params=training_params)



train_data = train.sample(frac=0.75)
predict_data = train.drop(train_data.index)

model.train(train_data,label_name=label)


In [ ]:
test=predict_data

In [ ]:
preds = model.predict(predict_data.drop(columns=label))

In [ ]:
test=test[label]

In [ ]:
preds=list(preds)

In [ ]:
test=list(test)

In [ ]:
test

In [ ]:

from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import precision_score,recall_score

In [ ]:
print("test LogLoss", round(log_loss(preds, test), 4))
print("test roc_auc_score", round(roc_auc_score(preds, test), 4))
print("test f1 score", round(f1_score(preds, test), 4))
print("test precision score", round(precision_score(preds, test), 4))
print("test recall score", round(recall_score(preds, test), 4))
print("test accuracy score", round(accuracy_score(preds, test), 4))